### Setup the environment to run juypter and django
Run this block of code first

In [ ]:
import os,sys
import django
from django.db import models
from django.db.models import Subquery, OuterRef
PWD = os.path.dirname(os.path.dirname(os.path.abspath("test-1.ipynb")))
print(f"PWD={PWD}\\")
sys.path.insert(0, f"{PWD}")

# wsqi.py
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "beer.settings")
#  Allow queryset filtering asynchronously when running in a Jupyter notebook
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()




### Get the race schedule for 2024

In [17]:
from nascar.models import Bet, Race, Track

# Race.objects.get(pk=1)
# https://docs.djangoproject.com/en/5.0/ref/models/expressions/
# print(Track.objects.get(pk=track_pk))

race_schedule = Race.objects.filter(race_date__gt="2024-03-01")
# print the actual sql code
print(race_schedule.query)
# race_schedule = Race.objects.filter(bet=OuterRef("pk"))
for rs in race_schedule:
    print(rs)

SELECT `race`.`date_created`, `race`.`user_id`, `race`.`date_updated`, `race`.`RACE_ID`, `race`.`RACE_NAME`, `race`.`RACE_DATE`, `race`.`Race Time`, `race`.`TRACK_ID`, `race`.`TV/RADIO` FROM `race` WHERE `race`.`RACE_DATE` > 2024-03-01 ORDER BY `race`.`RACE_DATE` ASC
Las Vegas Motor Speedway Pennzoil 400 - 2024-03-03
Phoenix Raceway Shriners Children's 500 - 2024-03-10
Bristol Motor Speedway NASCAR Cup Series at Bristol - 2024-03-17
Circuit Of The Americas NASCAR Cup Series at Circuit of the Americas - 2024-03-24
Richmond Raceway NASCAR Cup Series at Richmond - 2024-03-31
Martinsville Speedway NASCAR Cup Series at Martinsville - 2024-04-07
Texas Motor Speedway NASCAR Cup Series at Texas - 2024-04-14


### Get all the bets

In [21]:
# https://books.agiliq.com/projects/django-orm-cookbook/en/latest/query.html
sq = Race.objects.filter(track_fk=Subquery(race_schedule.values("track_fk"))).values(
    "track_fk"
)
# print(sq)
# show the sql query 
print(sq.query)
# get all bets
bets = Bet.objects.select_related("race_fk")
# print all bets 
for b in bets:
    print(f"{b.finish} {b.player_fk} {b.race_fk} {b.driver_fk}")

SELECT `race`.`TRACK_ID` FROM `race` WHERE `race`.`TRACK_ID` = (SELECT U0.`TRACK_ID` FROM `race` U0 WHERE U0.`RACE_DATE` > 2024-03-01 ORDER BY U0.`RACE_DATE` ASC) ORDER BY `race`.`RACE_DATE` ASC
32 Greg Daytona International Speedway Daytona 500 - 2024-02-18 22 - Joey Logano / Team Penske
14 Bob Daytona International Speedway Daytona 500 - 2024-02-18 9 - Chase Elliott / Hendrick Motorsports
28 Bob Atlanta Motor Speedway Ambetter Health 400 - 2024-02-25 22 - Joey Logano / Team Penske
32 Greg Atlanta Motor Speedway Ambetter Health 400 - 2024-02-25 5 - Kyle Larson / Hendrick Motorsports
1 Greg Las Vegas Motor Speedway Pennzoil 400 - 2024-03-03 5 - Kyle Larson / Hendrick Motorsports
10 Bob Las Vegas Motor Speedway Pennzoil 400 - 2024-03-03 24 - William Byron / Hendrick Motorsports
